<a href="https://colab.research.google.com/github/abhichiku18/docIQ/blob/main/dociq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install streamlit pyngrok pdfplumber faiss-cpu sentence-transformers transformers torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.2 MB/s eta 0:00:00


In [3]:
!pip install pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 68.1 MB/s eta 0:00:00


In [4]:
!pip install pdfplumber transformers sentence-transformers faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [10]:
import gradio as gr
import pdfplumber
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import random

# Initialize models
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# Global vars
chunks = []
index = None
stored_questions = []

def upload_and_summarize(file):
    global chunks, index
    try:
        if file.name.endswith('.pdf'):
            with pdfplumber.open(file.name) as pdf:
                text = ""
                for page in pdf.pages:
                    t = page.extract_text()
                    if t: text += t + "\n"
        elif file.name.endswith('.txt'):
            text = file.read().decode('utf-8')
        else:
            return "⚠ Unsupported file format."

        if not text.strip():
            return "⚠ Document appears empty or is scanned (needs OCR)."

        # Create chunks
        words = text.split()
        chunks = [' '.join(words[i:i+500]) for i in range(0, len(words), 500)]
        print(f"[DEBUG] Number of chunks: {len(chunks)}")

        # Build FAISS index
        embeddings = embed_model.encode(chunks, convert_to_numpy=True)
        index = faiss.IndexFlatL2(embeddings.shape[1])
        index.add(embeddings)

        # Summarize first ~1000 tokens
        summary = summarizer(text[:1024], max_length=150, min_length=30, do_sample=False)[0]['summary_text']
        return summary

    except Exception as e:
        return f"⚠ Failed to process: {e}"

def ask_anything(question):
    if not index or not chunks:
        return "**Error:** Please upload and process a document first."
    q_emb = embed_model.encode([question])
    D, I = index.search(np.array(q_emb), k=1)
    best_context = chunks[I[0][0]]
    result = qa_pipeline(question=question, context=best_context)
    return f"**Answer:** {result['answer']}\n\n_Justification:_ {best_context[:300]}..."

def start_challenge():
    global stored_questions
    if not chunks:
        return "⚠ Upload and process a document first.", "⚠ Upload and process a document first.", "⚠ Upload and process a document first."

    # Automatically generate 3 questions from real content:
    selected = random.sample(chunks, 3 if len(chunks) >= 3 else len(chunks))
    questions = []
    for c in selected:
        first_sentence = c.split('.')[0][:80]
        q = f"According to the document, what is discussed about: '{first_sentence}...'"
        questions.append(q)

    # If less than 3, pad
    while len(questions) < 3:
        questions.append("⚠ Not enough content to generate more questions.")

    stored_questions = questions
    return questions[0], questions[1], questions[2]

def evaluate(user1, user2, user3):
    if not stored_questions:
        return "⚠ Please generate questions first."
    answers = [user1, user2, user3]
    feedback = []
    for ans, ques in zip(answers, stored_questions):
        if "⚠" in ques:
            fb = f"**Q:** {ques}\n⚠ Cannot evaluate this question."
            feedback.append(fb)
            continue
        model_ans = ask_anything(ques).split("**Answer:** ")[-1].split("\n")[0]
        correct = "✅ Seems correct!" if model_ans.lower() in ans.lower() else "⚠ Could be improved."
        fb = f"**Q:** {ques}\n**Your Answer:** {ans}\n**Model's Answer:** {model_ans}\n**Feedback:** {correct}"
        feedback.append(fb)
    return "\n\n".join(feedback)

with gr.Blocks(title="📄 Smart Research Assistant") as demo:
    gr.Markdown("## 📄 Smart Research Assistant — single file, local model, no API key needed")

    with gr.Tab("Upload & Auto Summary"):
        file = gr.File(label="Upload PDF or TXT")
        summary = gr.Textbox(label="Summary (≤150 words)")
        file.upload(upload_and_summarize, inputs=file, outputs=summary)

    with gr.Tab("Ask Anything"):
        q = gr.Textbox(label="Your Question")
        a = gr.Markdown()
        q.submit(ask_anything, inputs=q, outputs=a)

    with gr.Tab("Challenge Me"):
        btn = gr.Button("Generate 3 Questions")
        q1 = gr.Textbox(label="Question 1", interactive=False)
        q2 = gr.Textbox(label="Question 2", interactive=False)
        q3 = gr.Textbox(label="Question 3", interactive=False)
        ua1 = gr.Textbox(label="Your Answer to Q1")
        ua2 = gr.Textbox(label="Your Answer to Q2")
        ua3 = gr.Textbox(label="Your Answer to Q3")
        feedback = gr.Markdown()

        btn.click(start_challenge, outputs=[q1, q2, q3])
        eval_btn = gr.Button("Submit My Answers")
        eval_btn.click(evaluate, inputs=[ua1, ua2, ua3], outputs=feedback)

demo.launch(share=True)


Device set to use cpu
Device set to use cpu


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3b0e9c72b7f207d2a1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
